# RAG 동작 테스트

**LLM**: ai-human-lab/EEVE-Korean_Instruct-10.8B-expo

**document**: PDFLoader, pdf 아무거나 google drive에 업로드하고 파일 이름 바꾼 뒤 실행

**text splitter**: RecursiveCharacterTextSplitter

**embedding**: jhgan/ko-sbert-nli

**vectorDB**: FAISS


In [ ]:
%%capture
!pip -q install langchain pypdf transformers sentence-transformers faiss-gpu
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes triton

## model, tokenizer load

### unsloth 사용안하고 로드

In [ ]:
#양자화에 필요한 패키지 설치
!pip install -q -U bitsandbytes accelerate
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 12.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 14.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
model_id = "ai-human-lab/EEVE-Korean_Instruct-10.8B-expo"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.18M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/722 [00:00<?, ?B/s]

...

model-00003-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

### unsloth 사용하고 로드
모델 로드 속도는 빠르나 랭체인 모듈을 사용할 때 문제 발생. 실행 X

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
if True:
    from unsloth import FastLanguageModel
    import torch

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "ai-human-lab/EEVE-Korean_Instruct-10.8B-expo", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        dtype = None,
        load_in_4bit = True,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.18M [00:00<?, ?B/s]

# eeve 모델 추론 테스트

In [ ]:
PROMPT_DICT = {
    "prompt_input": """다음은 작업을 설명하는 지침 사항과 추가적인 입력을 제공합니다. 요청을 적절하게 완료하는 응답을 한국어로 작성하세요.
### 지침 사항:
{instruction}

### 입력:
{input}

### 응답:""",
    "prompt_no_input": """다음은 작업을 설명하는 지침 사항입니다. 요청을 적절하게 완료하는 응답을 한국어로 작성하세요.
### 지침 사항:
{instruction}

### 응답:"""
}

# input이 없는 inference
if True:
  inputs = tokenizer(
      [PROMPT_DICT["prompt_no_input"].format(
          instruction="줄리어스 시저는 누구한테 배신당했나요?"
      )],
      return_tensors="pt"
  ).to("cuda")

# input이 있는 inference
if False:
  inputs = tokenizer(
      [PROMPT_DICT["prompt_input"].format(
          instruction="다음을 동물, 식물, 광물로 분류하십시오.",
          input="참나무, 구리, 코끼리"
      )],
      return_tensors="pt"
  ).to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens = 128,
    eos_token_id=tokenizer.eos_token_id, # EOS 토큰을 사용하여 명시적으로 출력의 끝을 지정.
    use_cache = True
)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
result = decoded_outputs[0]

print(result)  # print() 함수를 사용하지 않을 시 줄바꿈 문자 '\n' 반영 안됨


아래부터 RAG process

In [ ]:
!pip install langchain_community langchain_huggingface

In [ ]:
# # PDF 로더 설정
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough
loader = PyPDFLoader("/content/파일이름.pdf") # 왼쪽 메뉴에서 업로드 아이콘 눌러서 pdf 업로드한 후 파일 이름에 맞게 바꾸기
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(pages)

[HuggingFace Embeddings](https://python.langchain.com/api_reference/huggingface/embeddings/langchain_huggingface.embeddings.huggingface.HuggingFaceEmbeddings.html#langchain_huggingface.embeddings.huggingface.HuggingFaceEmbeddings)

In [ ]:
# 임베딩 모델 설정
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cuda'}
# normalize_embeddings: 벡터의 크기를 1로 만들어 코사인 유사도 계산할 때 내적 사용 가능
encode_kwargs = {'normalize_embeddings': True}
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

[HuggingFace Pipeline](https://python.langchain.com/api_reference/huggingface/llms/langchain_huggingface.llms.huggingface_pipeline.HuggingFacePipeline.html)

In [ ]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    return_full_text=True,
    max_new_tokens=100,
)

llm_eeve = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt_template = """
### [INST]
주어진 정보에 기반해 질문에 한국어로 답하세요.
{context}

### 사용자의 질문:
{question}

[/INST]
"""

# Create prompt from prompt template
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
#from langchain.chains import LLMChain
llm_chain = prompt | llm_eeve

In [ ]:
print(model)

In [ ]:
print(tokenizer)

In [ ]:
# Example context and question for testing
context = """
유명한 유튜버이자 블로거인 제니는 주로 패션과 뷰티 관련 콘텐츠를 다루고 있습니다. 그녀는 또한 다양한 브랜드와 협업하여 리뷰 콘텐츠를 만들며, 구독자들에게 매우 신뢰받는 인플루언서입니다. 최근 제니는 구독자들에게 자연스러운 메이크업 트렌드에 대해 소개했습니다.
"""

question = "제니가 최근 소개한 메이크업 트렌드는 무엇인가요?"

# Run the llm_chain
response = llm_chain.invoke({"context": context, "question": question})

# Output the response
print("LLM Response:")
print(response)


LLM Response:

### [INST]
주어진 정보에 기반해 질문에 한국어로 답하세요.

유명한 유튜버이자 블로거인 제니는 주로 패션과 뷰티 관련 콘텐츠를 다루고 있습니다. 그녀는 또한 다양한 브랜드와 협업하여 리뷰 콘텐츠를 만들며, 구독자들에게 매우 신뢰받는 인플루언서입니다. 최근 제니는 구독자들에게 자연스러운 메이크업 트렌드에 대해 소개했습니다.


### 사용자의 질문:
제니가 최근 소개한 메이크업 트렌드는 무엇인가요?

[/INST]

제니가 최근 소개한 메이크업 트렌드는 자연스러운 메이크업입니다.


[FAISS](https://python.langchain.com/api_reference/community/vectorstores/langchain_community.vectorstores.faiss.FAISS.html)

[VectorStore](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html#langchain_core.vectorstores.base.VectorStore)

[VectorStore.as_retriever()](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html#langchain_core.vectorstores.base.VectorStore.as_retriever): Return VectorStoreRetriever initialized from this VectorStore. 파라미터 설정 가능

[VectorStoreRetriever](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStoreRetriever.html#langchain_core.vectorstores.base.VectorStoreRetriever)

In [ ]:
vectorstore = FAISS.from_documents(texts, hf_embeddings)

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 3}
)

# RAG 체인 생성
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)


In [ ]:
# 토크나이저와 모델의 최대 시퀀스 길이 확인
print(tokenizer.model_max_length)

아래의 코드가 몇 분째 결과가 나오지 않습니다

In [ ]:
# 경고 무시 설정
#import warnings
#warnings.filterwarnings('ignore')

# 질문에 대한 답변 생성
result = rag_chain.invoke("What is SAMSum dataset?")

# 결과 출력
print(result['context'])
print(f"\n답변: {result['text']}")

#### FAISS Test

In [ ]:
!pip install faiss-cpu # For CPU Installation
!pip install langchain langchain_community langchain_huggingface pypdf

In [ ]:
!pip install chromadb tiktoken transformers sentence_transformers openai langchain pypdf

In [ ]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough

loader = PyPDFLoader("/content/filename.pdf")
pages = loader.load_and_split()

# split it into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0, length_function = tiktoken_len)
docs = text_splitter.split_documents(pages)

from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
ko = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
db = FAISS.from_documents(docs, ko)

In [ ]:
query = "SAMSum 데이터셋에 대해 알려줘."
docs = db.similarity_search(query)
print(docs[0].page_content)

In [ ]:
docs_and_scores = db.similarity_search_with_score(query)
docs_and_scores

In [ ]:
query = "SAMSum 데이터셋에 대해 알려줘."
docs = db.similarity_search_with_relevance_scores(query, k=3)


print("질문: {} \n".format(query))
for i in range(len(docs)):
    print("{0}번째 유사 문서 유사도 \n{1}".format(i+1,round(docs[i][1],2)))
    print("-"*100)
    print(docs[i][0].page_content)
    print("\n")
    print(docs[i][0].metadata)
    print("-"*100)